# Book-Crossing Recommendation System

**Fellowship AI challenge**  
*Tom McKenzie*  
*July 2018*  

**Aim:** The aim of this project is to build a recommendation engine using the dataset(s) from Book-Crossing, which includes user reviews.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

## Import the dataset

In [2]:
# The dataset is comprised of three separate tables: BX-Users, BX-Books, and BX-Book-Ratings.
# Each is separated by semi-colons, and the first row contains the column headers.
users = pd.read_csv('book_crossing_dataset/BX-Users.csv', sep=';')
books = pd.read_csv('book_crossing_dataset/BX-Books.csv', sep=';')
book_ratings = pd.read_csv('book_crossing_dataset/BX-Book-Ratings.csv', sep=';')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Note: The 'BX-Books' csv had multiple cases of semi-colons in the book titles which were manually cleaned to ensure proper dataframe importing. Mostly were changed to colons or commas. Also, the symbol '&amp;' appeared a lot (presumably an ampersand character), which was changed to just '&amp'.

In [3]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
book_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [7]:
print(f'There are {len(users)} users, {len(books)} books, and {len(book_ratings)} ratings in these datasets.')

There are 278858 users, 271379 books, and 1149780 ratings in these datasets.


I don't think I need the image links in the 'books' dataframe

In [8]:
books.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], inplace=True)

In [9]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp Company


Can probably expand out the 'Location' columns in the users df.

In [71]:
# def split_location(x):
#     s = x.split(',')
#     if len(s) == 3:
#         city = s[0]
#         state = s[1]
#         country = s[2]
#     else:
#         city, state, country = None, None, None
    
#     return pd.Series([city, state, country], index=['City', 'State', 'Country'])

# users[['City', 'State', 'Country']] = users['Location'].apply(split_location)

In [10]:
# Note: Used Pandas Series.str.split method as it has an 'expand' parameter which can handle None cases
# users[['City', 'State', 'Country']] = users['Location'].str.split(',', expand=True).apply(lambda x: x.str.strip())
# users = users.drop(columns=['Location'])
#users_mini = users[['User-ID', 'Location']][:50]
users[['city', 'state', 'count']] = users['Location'].str.split(',', 3, expand=True)
users

ValueError: Columns must be same length as key

In [11]:
pd.__version__

'0.22.0'